In [1]:
import pandas as pd
from pathlib import Path
from sqlalchemy.orm import Session
from sqlalchemy.sql import func, or_
from database.queries import join_all_tables
from database.builder import RLCSParser, parse_directory_structure
from database.models import *

In [2]:
rlcs_db = RLCSParser(database_url="sqlite:///../data/rlcs23.db", parsed_dir="D:/parsed")

In [3]:
with Session(rlcs_db.engine) as session:
    query = join_all_tables(session.query(
        Replay.hash.label("replay_hash"),
        func.coalesce(
            EventFromMatch.name,
            EventFromStage.name
        ).label("event_name"),
        func.coalesce(
            SplitFromEvent.name,
            SplitFromRegion.name
        ).label("split_name"),
        func.coalesce(
            StageFromRound.name,
            StageFromRound.name
        ).label("stage_name")
    ))\
        .filter(
            or_(SplitFromEvent.name == "Winter", SplitFromRegion.name == "Winter"),
            Season.name == "RLCS 22-23"
        )\
        .distinct(Replay.hash)
    
    replay_df = pd.read_sql_query(query.statement, rlcs_db.engine)
replay_df

,replay_hash,event_name,split_name,stage_name
0,0023001c1e588c64e3dc7cf42867c904b7df13987452da...,Qualifiers,Winter,Swiss
1,0024bb9e69dd5792e467ad1354dee9ee56ffd39f018e74...,Main Event,Winter,Groups
2,00273b957f8bf3461e302bc33575728cfae79d53a061fb...,Main Event,Winter,Groups
3,0049653a5d03bf166a572a3779727bac1390f19fd5daf8...,Main Event,Winter,Groups
4,005f5834661f021d41f41dcc6fa8aa0e62d3e624cc3456...,Main Event,Winter,Playoffs
...,...,...,...,...
1574,fee943fa10db5fb41797b1e63d28e7ca0395c108911dd2...,Main Event,Winter,Groups
1575,ff0ede3e47eea3db4623b9930e64972be364952153dfab...,Main Event,Winter,Groups
1576,ffb6b0ff572860a1dc115f4fb66ff48f1dbdce91d42220...,Main Event,Winter,Groups
1577,ffe33f898a848380037d97428859d7fb561c4c01232a0b...,Main Event,Winter,Playoffs


In [4]:
with Session(rlcs_db.engine) as session:
    query = session.query(Stage)
    replay_df = pd.read_sql_query(query.statement, rlcs_db.engine)
replay_df

,id,event_id,name,type
0,1,1,Swiss,None
1,2,2,Playoffs,None
2,3,2,Upper,None
3,4,2,Lower,None
4,5,3,Playoffs,None
5,6,3,Groups,None
6,7,4,Playoffs,None
7,8,4,Groups,None
8,9,5,Swiss,None
9,10,6,Swiss,None


In [5]:
replays = []
for replay_path in Path("D:/rlcs/RLCS 22-23").glob("**/*.replay"):
    if replay_path.is_file():
        replay_dict = parse_directory_structure(replay_path)
        replays.append(replay_dict)

replays_df = pd.DataFrame(replays)
replays_df

,raw_path,season_name,split_name,event_name,stage_name,round_name,match_name,group_name,region_name
0,D:\rlcs\RLCS 22-23\Worlds\Wilcard\Swiss\Round ...,RLCS 22-23,Worlds,Wilcard,Swiss,Round 5,MST vs G1,NaN,NaN
1,D:\rlcs\RLCS 22-23\Worlds\Wilcard\Swiss\Round ...,RLCS 22-23,Worlds,Wilcard,Swiss,Round 5,MST vs G1,NaN,NaN
2,D:\rlcs\RLCS 22-23\Worlds\Wilcard\Swiss\Round ...,RLCS 22-23,Worlds,Wilcard,Swiss,Round 5,MST vs G1,NaN,NaN
3,D:\rlcs\RLCS 22-23\Worlds\Wilcard\Swiss\Round ...,RLCS 22-23,Worlds,Wilcard,Swiss,Round 5,MST vs G1,NaN,NaN
4,D:\rlcs\RLCS 22-23\Worlds\Wilcard\Swiss\Round ...,RLCS 22-23,Worlds,Wilcard,Swiss,Round 5,MST vs G1,NaN,NaN
...,...,...,...,...,...,...,...,...,...
5402,D:\rlcs\RLCS 22-23\Fall\Major\Main Event\Playo...,RLCS 22-23,Fall,Main Event,Playoffs,Finals,MST vs GGM1,NaN,NaN
5403,D:\rlcs\RLCS 22-23\Fall\Major\Main Event\Playo...,RLCS 22-23,Fall,Main Event,Playoffs,Finals,MST vs GGM1,NaN,NaN
5404,D:\rlcs\RLCS 22-23\Fall\Major\Main Event\Playo...,RLCS 22-23,Fall,Main Event,Playoffs,Finals,MST vs GGM1,NaN,NaN
5405,D:\rlcs\RLCS 22-23\Fall\Major\Main Event\Playo...,RLCS 22-23,Fall,Main Event,Playoffs,Finals,MST vs GGM1,NaN,NaN
